In [1]:
import os 
import json
import pandas as pd
import spacy
import re
import string
from collections import defaultdict
import warnings
import numpy as np

warnings.filterwarnings('ignore')

2024-04-01 23:09:09.473928: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-01 23:09:10.145767: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-01 23:09:11.417131: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-01 23:09:17.117648: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Covert the gold training chemo data as temporal_nli format 
1. from gold pair-wise file to add nth_sent for both chemo and timex mentions
2. at baseline pair-wise pred output, add start_ind, end_ind, nth_sent for each mentions
3. generate candiatate pair from 2, mark the pair as negative if it is not at 1 (need index to compare)
4. for each nth sent has potential rel, generate the dct with args and rel 
##### 5. repeat for single patient_file as dct (count positive and negative rels  the distribution, if little negative, need to think way to improve)
##### 6. repeat for cancer_mode with all patients and files as json file
##### 7. repeat for all cancer by combing training and dev json file
##### 8. random choose x% patient from 6 and merge as a x%_sample_json file



##### 3. save file as json for each cancer train
##### 4. save file as json for all cancer train

In [ ]:
##### 1. from gold pair-wise file to add nth_sent for both chemo and timex mentions
cancer, mode = "breast", "dev"
gold_dct_file = f"/users/the/NER_MTB/0_{cancer}_{mode}_gold_dct.json"
# here has been replace /n to space
note_dir = os.path.join("/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/input/change_n2space", \
                        f"input_{cancer}_{mode}", "Patient_Notes", f"{cancer}", f"{mode}")
unsummarized_file = f"/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/output/change_n2space/unsummarized_{cancer}_{mode}_output.tsv"
df = pd.read_csv(unsummarized_file, delimiter="\t")

def find_loc(a, b):
    # a: long one
    a_clean = ''.join(a.split())
    b_clean = ''.join(b.split())
    index = a_clean.find(b_clean)
    s, out = 0, []
    for i, char in enumerate(a):
        if char == " " or char == "\t":
            continue 
        s+=1 
        if s == index + 1: 
            out.append(i)
            
        if s == index + len(b_clean):
            out.append(i)
    # if len(out) == 1: 
    #     end, start = None, None

    # elif len(out) == 2:  # else
    if len(out) == 2:
        end, start = out[0], out[1]+1
        return end, start
    else: 
        return None, None

def _get_seindexs_single_note(string, spacy_better):
    """Return the inedx of sent and sent_lst after doc"""
    doc = spacy_better(string)
    seindexs_single_note = [] 
    sent_lst = []
    for i, sent in enumerate(doc.sents):
        seindexs_single_note.append((i, sent.start_char, sent.end_char))
        sent_lst.append(sent.text)
    return seindexs_single_note, sent_lst

def _find_nthind(tuple_list, item_start_ind, num_pre9_tokens):
    for tup in tuple_list:
        if tup[1] <= (int(item_start_ind)-num_pre9_tokens) <= tup[2]:
            return tup[0]
    print("something wrong with this case") # TO-DO: need to fix here, run the below code first since little error msgs
    return None  # Return None if no match is found # across the sentences

def _find_surrounding_text(string):
    tlink_inst = string
    e_pattern = r'<e>\s*(.*?)\s*</e>'
    matches = re.findall(e_pattern, tlink_inst)
    return matches

def _find_time_surrounding_text(string):
    tlink_inst = string
    t_pattern = r'<t>\s*(.*?)\s*</t>'
    matches = re.findall(t_pattern, tlink_inst)
    return matches

def match_ignore_spaces(a, b):
    # Create a pattern for string a with optional spaces between each character
    pattern_a = r'\b{}\b'.format(r'\s*'.join(re.escape(word) for word in a.split()))
    # Find the matched pattern in string b
    match = re.search(pattern_a, b)
    if match:
        return match.start(), match.end()
    else:
        return None, None


def _find_time_word_index(original_text, given_string, word):
    original_text_processed = original_text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    # original_text_processed = re.sub(r'\s+', ' ', original_text_processed)
    word = word.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))

    match_s, match_e = match_ignore_spaces(word, original_text_processed)
    # print(match_s, match_e)
    return match_s, match_e


def _find_chemo_word_index(original_text, given_string, word):
    # print(len(word), word)
    # given_string_processed = given_string.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    original_text_processed = original_text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    # original_text_processed = re.sub(r'\s+', ' ', original_text_processed)
    word = word.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    # print("2:", len(word), word)
    # word = re.sub(r'\s+', ' ', word)
    # print(word, "***", original_text_processed)
    
    # edge case
    # if word == "t   c": 
    #     word = "t c"
    #     match = re.search(r'{}'.format(re.escape(word)), original_text_processed)
    # elif word == "T   C":
    #     word = "T C"
    #     match = re.search(r'{}'.format(re.escape(word)), original_text_processed)
    # elif word == "alpha   2b interferon":
    #     word = "alpha 2b interferon"
    #     match = re.search(r'{}'.format(re.escape(word)), original_text_processed)
    # elif word == "interleukin   2": 
    #     word = "interleukin 2"
    #     match = re.search(r'{}'.format(re.escape(word)), original_text_processed) 
    # else: 
    #     match = re.search(r'\b{}\b'.format(re.escape(word)), original_text_processed)
    # if match:
    #     # print(match.start(), match.end())
    #     return match.start(), match.end()
    # else:
    #     return None, None
    
def same_characters_in_order(s1, s2):
    # Remove punctuation from both strings
    s1_cleaned = ''.join(char for char in s1 if char not in string.punctuation)
    s2_cleaned = ''.join(char for char in s2 if char not in string.punctuation)
    # Compare the cleaned strings
    return s1_cleaned, s2_cleaned

def decrease_space_before_punctuation(text):
    # Define the regular expression pattern
    pattern = r'(?<=\w)\s+(?=\W)'
    # Replace multiple spaces with a single space before punctuation
    cleaned_text = re.sub(pattern, ' ', text)
    return cleaned_text

def _find_nth(lst, chemo_or_timex):
    "Used the loc from raw to find where sent it is in Doc"
    nth_sent_lst = []
    if len(lst)!=0:
        for i, chemo in enumerate(lst):
            if chemo_or_timex == "chemo":
                item_start_ind, item_end_ind = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["span"][i][0], gold_dct[pat_id][filename]["pair_wise"]["chemo"]["span"][i][1]
            else:
                item_start_ind, item_end_ind = gold_dct[pat_id][filename]["pair_wise"]["time"]["span"][i][0], gold_dct[pat_id][filename]["pair_wise"]["time"]["span"][i][1]

            # print(item_start_ind, item_end_ind)
            for sent_id, sent in enumerate(doc.sents):
                # print(sent.start_char, sent.end_char)
                if sent.start_char <= item_start_ind-num_pre9_tokens< sent.end_char: 
                    for token in sent:
                        if token.idx <= item_start_ind-num_pre9_tokens< (token.idx+len(token.text)):
                            # print("Token:", token)
                            # print("Sentence:", sent)
                            # print("Sentence ID:", sent_id)
                            print(pat_id, filename)
                            nth_sent_lst.append(sent_id)
                        else: 
                            print("Token:", token)
                            print("Sentence:", sent)
                            print("Sentence ID:", sent_id) 

    print(len(lst), len(nth_sent_lst))
    assert len(lst) == len(nth_sent_lst)
    return nth_sent_lst

def _find_tsv_sentences_in_range(doc, start_index, end_index):
    "Given nth sents of chemo and timex, return the context"
    sentences_in_range = []
    for ind, sent in enumerate(doc.sents):
        if ind >= start_index and ind < end_index:
            sentences_in_range.append(sent)
    return sentences_in_range

def _find_sentences_in_range(doc, start_index, end_index):
    "Given nth sents of chemo and timex, return the context"
    sentences_in_range = []
    for ind, sent in enumerate(doc.sents):
        if ind == start_index:
            sentences_in_range.append(sent.text)
            ind += 1
            while ind < end_index:
                sentences_in_range.append(list(doc.sents)[ind].text)
                ind += 1
            return sentences_in_range

def _df_find_nth(lst, match_start_lst, match_end_lst):
    "Used the loc from raw to find where sent it is in Doc"
    nth_sent_lst = []
    ith_start_lst, ith_end_lst = [], []
    if len(lst)!=0:
        for i, chemo in enumerate(lst):
            item_start_ind, item_end_ind = match_start_lst[i], match_end_lst[i]
            if item_end_ind is None or item_start_ind is None:
                print("skip here: ", item_start_ind, item_end_ind )
                item_end_ind, item_start_ind = -1, -1
                # continue

            len_ith = item_end_ind - item_start_ind

            # print("chemo: ", item_start_ind, item_end_ind)
            for sent_id, sent in enumerate(doc.sents):
                # print(sent.start_char, sent.end_char)
                if sent.start_char <= item_start_ind< sent.end_char: 
                    # print("yes")
                    ith_start_ind = int(item_start_ind - sent.start_char)
                    ith_end_ind = item_start_ind + len_ith
                    for token in sent:
                        if token.idx <= item_start_ind< (token.idx+len(token.text)):
                            # print("Token:", token)
                            # print("Sentence:", sent)
                            # print("Sentence ID:", sent_id)
                            nth_sent_lst.append(int(sent_id))
                            ith_start_lst.append(ith_start_ind)
                            ith_end_lst.append(ith_end_ind)
                            # print("\n")
                # else:
                #     nth_sent_lst.append(-1)
                #     ith_start_lst.append(-1)
                #     ith_end_lst.append(-1)
    print("lst", lst)
    print("nth_sent_lst", nth_sent_lst)
    diff = len(lst) - len(nth_sent_lst)
    if len(lst) > len(nth_sent_lst):
        while diff > 0:
            nth_sent_lst.append(-1)
            diff -= 1
    assert len(lst) == len(nth_sent_lst)
    return nth_sent_lst, ith_start_lst, ith_end_lst
    
with open(gold_dct_file, "r") as infile: 
    gold_dct = json.load(infile)

spacy_better = spacy.load("en_core_web_sm")

pos_rel = []
final_df = pd.DataFrame()
cnt = 0
# for pat_id in ["patient47"]:
for pat_id in gold_dct.keys():
    note_sub_dir = os.path.join(note_dir, pat_id)
    medium_df = pd.DataFrame()
    for filename in gold_dct[pat_id].keys():
    # for filename in ["report040_NOTE"]:

        # raw_note = gold_dct[pat_id][filename]["raw_note"]
        pat_filename = f"{pat_id}_{filename}.txt"
        raw_note_file = os.path.join(note_sub_dir, pat_filename)
        with open(raw_note_file, "r") as infile: 
            lines = infile.readlines()
        num_pre9_tokens = len(''.join(lines[:9]))
        lines = lines[9:]
        clean_note = "".join(lines)
        doc = spacy_better(clean_note)
        # print(f"len(doc.sents), ", len(list(doc.sents)))

        chemo_lst, timex_lst = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["ment"], gold_dct[pat_id][filename]["pair_wise"]["time"]["ment"]
        
        # print(chemo_lst, timex_lst)
        chemo_nth_sent = _find_nth(chemo_lst, "chemo")
        timex_nth_sent = _find_nth(timex_lst, "timex")
        
        gold_dct[pat_id][filename]["pair_wise"]["chemo"]["nth_sent"] = chemo_nth_sent
        gold_dct[pat_id][filename]["pair_wise"]["time"]["nth_sent"] = timex_nth_sent
        gold_dct[pat_id][filename]["num_pre9_tokens"] = num_pre9_tokens

        # create positive instance
        gold_pair = gold_dct[pat_id][filename]["chemo_time_rel"]["rel_id"]
        if len(gold_pair) == 0:
            continue
        gold_sid_pair = gold_dct[pat_id][filename]["chemo_time_rel"]["source_id"]
        gold_tid_pair = gold_dct[pat_id][filename]["chemo_time_rel"]["target_id"]

        num_pre9_tokens = gold_dct[pat_id][filename]["num_pre9_tokens"]

        rel_type_lst = gold_dct[pat_id][filename]["chemo_time_rel"]["rel_type"]

        chemo_ment_lst, time_ment_lst = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["ment"], gold_dct[pat_id][filename]["pair_wise"]["time"]["ment"]
        chemo_span_lst, time_span_lst = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["span"], gold_dct[pat_id][filename]["pair_wise"]["time"]["span"]
        chemo_nth_sent_lst, time_nth_sent_lst = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["nth_sent"], gold_dct[pat_id][filename]["pair_wise"]["time"]["nth_sent"]
        
        chemo_id, time_id = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["ment_id"], gold_dct[pat_id][filename]["pair_wise"]["time"]["ment_id"]
        combination = []
        for i in range(len(gold_pair)):
            pos_ins = defaultdict(dict)
            source_id, target_id = gold_sid_pair[i], gold_tid_pair[i]
            if source_id in chemo_id: 
                chemo_ind = chemo_id.index(source_id) 
                chemo_ment = chemo_ment_lst[chemo_ind]
                chemo_start, chemo_end = chemo_span_lst[chemo_ind][0], chemo_span_lst[chemo_ind][1]
                chemo_nth_sent = chemo_nth_sent_lst[chemo_ind]
                if target_id not in time_id:
                    print(pat_id, filename)
                time_ind = time_id.index(target_id)
                time_ment = time_ment_lst[time_ind]
                time_start, time_end = time_span_lst[time_ind][0], time_span_lst[time_ind][1]
                time_nth_sent = time_nth_sent_lst[time_ind]
            else: 
                time_ind = time_id.index(source_id) 
                time_ment = time_ment_lst[time_ind]
                time_start, time_end = time_span_lst[time_ind][0], time_span_lst[time_ind][1]
                time_nth_sent = time_nth_sent_lst[time_ind]
                chemo_ind = chemo_id.index(target_id) 
                chemo_ment = chemo_ment_lst[chemo_ind]
                chemo_start, chemo_end = chemo_span_lst[chemo_ind][0], chemo_span_lst[chemo_ind][1]
                chemo_nth_sent = chemo_nth_sent_lst[chemo_ind]

            pos_ins["filename"] = f"{pat_id}_{filename}"
            pos_ins["statement"] = "Does the arg1 and arg2 has relationship?"
            pos_ins["num_pre9_tokens"] = num_pre9_tokens
            pos_ins["rel_type"] = rel_type_lst[i] 
            pos_ins["label"] = "positive"
            pos_ins["arg1"] = {
                "mention":chemo_ment,
                "start_paper_ind":chemo_start,
                "end_paper_ind":chemo_end,
                "nth_sent":chemo_nth_sent
            }
            pos_ins["arg2"] = {
                "mention":time_ment,
                "start_paper_ind":time_start,
                "end_paper_ind":time_end,
                "nth_sent":time_nth_sent
            }
            if int(chemo_nth_sent) > int(time_nth_sent):
                start_nth_sent = time_nth_sent
                end_nth_sent = chemo_nth_sent
            else: 
                start_nth_sent = chemo_nth_sent
                end_nth_sent = time_nth_sent
            pos_ins["context"] = _find_sentences_in_range(doc, start_nth_sent, end_nth_sent+1) 
        
            pos_rel.append(pos_ins)


 
        # 2 generate negative sample
        chemo_start_ind_tlinkinst, chemo_end_ind_tlinkinst = [], []
        nth_sent = []
        chemo_lst = []
        time_start_ind_tlinkinst, time_end_ind_tlinkinst = [], []
        time_nth_sent = []
        raw_time_lst = []
        match_sents = []

        selected_i = []
        patient_file = df.loc[df["note_name"] == pat_id + "_" + filename]
        tsv_df = pd.DataFrame()
        selected_row = []
        for i, row in patient_file.iterrows():
            tlink = row["tlink"]
            tlink_inst = row["tlink_inst"]
            # if row["note_name"] == "patient01_report002_NOTE":
            #     print(row["tlink_inst"])
            # negtaive sample, rel is none but normed_timex and chemo_text is not NONE
            if tlink == "none" and row["chemo_text"] != "none" and row["normed_timex"] != "none":
                matches = _find_surrounding_text(tlink_inst)
                time_matches = _find_time_surrounding_text(tlink_inst)

                selected_row.append(row)
                
                # if print out, issue with your code
                if len(matches) == 0 or len(time_matches) == 0:
                    if len(matches) == 0:
                        print("chemo: yese", matches)
                    elif len(time_matches) == 0:
                        print("1time: ")

                clean_match = matches[0].translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
                time_clean_match = time_matches[0].translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
                
                chemo_lst.append(clean_match)
                raw_time_lst.append(time_clean_match) # not exact raw time, but if use the index it is fine
                
                start_ind, end_ind = find_loc(clean_note, clean_match)
                time_start_ind, time_end_ind = _find_time_word_index(clean_note, tlink_inst, time_clean_match)


                # if time_start_ind == None and time_end_ind == None:
                #     print()
                #     time_clean_match = decrease_space_before_punctuation(time_clean_match)
                #     time_start_ind, time_end_ind = _find_time_word_index(clean_note, tlink_inst, time_clean_match)
             
                chemo_start_ind_tlinkinst.append(start_ind)
                chemo_end_ind_tlinkinst.append(end_ind)
                time_start_ind_tlinkinst.append(time_start_ind)
                time_end_ind_tlinkinst.append(time_end_ind)

            #     temp_time1 = "".join(clean_note[time_start_ind:time_end_ind].split())
            #     temp_time2 = "".join(time_matches[0].split())
            #     temp_time1, temp_time2 = same_characters_in_order(temp_time1, temp_time2)

            #     if clean_note[start_ind:end_ind] != matches[0] or temp_time1 !=temp_time2:
            #         if clean_note[start_ind:end_ind] != matches[0]:
            #             print("chemo: no", pat_id, filename, start_ind, end_ind, matches[0])
                        
            #         elif clean_note[time_start_ind:time_end_ind] !=time_matches[0]:
            #             print("2time: ") 
            #             print(row["chemo_annotation_id"], time_start_ind, time_end_ind, temp_time2, temp_time1, time_clean_match)

            # else:
                # chemo_start_ind_tlinkinst.append(None)
                # chemo_end_ind_tlinkinst.append(None)
                # time_start_ind_tlinkinst.append(None)
                # time_end_ind_tlinkinst.append(None)
                # raw_time_lst.append(None)
                continue
   
        nth_sent_item, ith_start_item, ith_end_item = _df_find_nth(chemo_lst, chemo_start_ind_tlinkinst, chemo_end_ind_tlinkinst) 
        time_nth_sent_item, time_ith_start_item, time_ith_end_item = _df_find_nth(raw_time_lst, time_start_ind_tlinkinst, time_end_ind_tlinkinst)

        tsv_df["chemo_tlinkinst_start"] = chemo_start_ind_tlinkinst
        tsv_df["chemo_tlinkinst_end"] = chemo_end_ind_tlinkinst
        tsv_df["nth_sent"] = nth_sent_item
        # tsv_df["ith_start"] = ith_start_item
        # tsv_df["ith_end"] = ith_end_item
        tsv_df["time_tlinkinst_start"] = time_start_ind_tlinkinst
        tsv_df["time_tlinkinst_end"] = time_end_ind_tlinkinst
        tsv_df["time_nth_sent"] = time_nth_sent_item
        tsv_df["time_ith_start"] = time_ith_start_item
        tsv_df["time_ith_end"] = time_ith_end_item
        tsv_df["raw_time"] = raw_time_lst

        sent_range_lst = []
        for i in range(len(time_nth_sent_item)):
            if int(nth_sent_item[i]) > int(time_nth_sent_item[i]):
                start_nth_sent = int(time_nth_sent_item[i])
                end_nth_sent = int(nth_sent_item[i])
            else: 
                start_nth_sent = int(nth_sent_item[i])
                end_nth_sent = int(time_nth_sent_item[i])
            sent_range = _find_tsv_sentences_in_range(doc, start_nth_sent, end_nth_sent+1)
            # print("ent_range: ", sent_range)
            sent_range_lst.append(sent_range) 
        tsv_df["match_sents"] = sent_range_lst
        selected_pat_file = patient_file.iloc[selected_i]
        selected_pat_file = pd.DataFrame(columns=["DCT", "patient_id", "chemo_text", "chemo_annotation_id", "normed_timex", "timex_annotation_id", "tlink", "note_name", "tlink_inst"], data=selected_row)
        new_df = pd.concat([selected_pat_file.reset_index(), tsv_df.reset_index()], axis=1)
        medium_df = pd.concat([new_df, medium_df])
    final_df = pd.concat([final_df, medium_df])
final_df = final_df.reset_index()

final_df.to_csv(f"./{cancer}_{mode}_neg_rel.tsv", sep="\t", index=False)

with open(f"./{cancer}_{mode}_pos_rel.json", 'w') as outfile:
    json.dump(pos_rel, outfile, indent=4)


In [ ]:
final_df

In [4]:
unsummarized_file = f"/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/output/change_n2space/unsummarized_{cancer}_{mode}_output.tsv"
df = pd.read_csv(unsummarized_file, delimiter="\t")
df.columns

Index(['DCT', 'patient_id', 'chemo_text', 'chemo_annotation_id',
       'normed_timex', 'timex_annotation_id', 'tlink', 'note_name',
       'tlink_inst'],
      dtype='object')

In [ ]:
note_dir = os.path.join("/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/input/change_n2space", \
                        f"input_{cancer}_{mode}", "Patient_Notes", f"{cancer}", f"{mode}")

unsummarized_file = f"/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/output/change_n2space/unsummarized_{cancer}_{mode}_output.tsv"
df = pd.read_csv(unsummarized_file, delimiter="\t")

tsv_lst = []
pat_filename = np.unique(final_df["note_name"]).tolist()
pat_id = pat_filename
for uni_pat_filename in pat_filename:
    single_dct = defaultdict(dict)
    single_df = final_df[final_df["note_name"] == uni_pat_filename]
    print(single_df.columns)
    for i, row in single_df.iterrows():
        print(i, row)
        if abs(row["nth_sent"]-row["time_nth_sent"]) <= 1:
            single_dct["filename"] = uni_pat_filename
            single_dct["statement"] = "Does the arg1 and arg2 has relationship?"
            single_dct["arg1"] = {
                "mention": row["chemo_text"],
                # "chemo_tlinkinst_start": int(row["chemo_tlinkinst_start"]),
                # "chemo_tlinkinst_end": int(row["chemo_tlinkinst_end"]),
                "nth_sent": int(row["nth_sent"]),
                # "ith_start": int(row["ith_start"]),
                # "ith_end": int(row["ith_end"]),
            }
            single_dct["arg2"] = {
                "mention": row["raw_time"],
                "time_tlinkinst_start": int(row["time_tlinkinst_start"]),
                "time_tlinkinst_end": int(row["time_tlinkinst_end"]),
                "time_nth_sent": int(row["time_nth_sent"]),
                # "time_ith_start": int(row["time_ith_start"]),
                # "time_ith_end": int(row["time_ith_end"]),
            }
            
            pat_id = uni_pat_filename.split("_")[0]
            note_sub_dir = os.path.join(note_dir, pat_id)
            pat_filename_txt = uni_pat_filename+".txt"
            raw_note_file = os.path.join(note_sub_dir, pat_filename_txt)

            with open(raw_note_file, "r") as infile: 
                lines = infile.readlines()
            num_pre9_tokens = len(''.join(lines[:9]))
            single_dct["num_pre9_tokens"] = num_pre9_tokens 

            lines = lines[9:]
            clean_note = "".join(lines)
            doc = spacy_better(clean_note)
            
            if int(row["nth_sent"]) < int(row["time_nth_sent"]):
                start_sent_ind = int(row["nth_sent"])
                end_start_ind = int(row["time_nth_sent"])
            else: 
                start_sent_ind = int(row["time_nth_sent"])
                end_start_ind = int(row["nth_sent"])
            
            temp = []
            for i, sent in enumerate(doc.sents):
                if i<= end_start_ind and i >= start_sent_ind:
                    temp.append(sent.text)
            single_dct["context"] = " ".join(temp)
            # temp =" ".join(temp)
            # single_dct[]
            tsv_lst.append(single_dct)
tsv_lst

In [6]:
neg_lst = []

tsv_filename_lst = [x["filename"] for x in tsv_lst]
# json exclude pos pair in gold file 
for pair in pos_rel:
    pat_filename = pair["filename"]
    if pat_filename in tsv_filename_lst: 
        tsv_fill_lst = [i for i in tsv_lst if i["filename"]==pat_filename]
        for uni in tsv_fill_lst:
            # print(uni)
            # print(pair)
            if ("".join(uni["arg1"]["mention"]) == "".join(pair["arg1"]["mention"])) and \
                ("".join(uni["arg2"]["mention"]) == "".join(pair["arg2"]["mention"])) and \
                    (uni["arg1"]["nth_sent"] == pair["arg1"]["nth_sent"]) and \
                         (uni["arg2"]["time_nth_sent"] == pair["arg2"]["nth_sent"]):
                    print("found one pos rel in the tsv file ...")
                    continue 
            else:
                uni["label"] = "negative"
                if uni in neg_lst:
                    continue
                else:
                    neg_lst.append(uni)

found one pos rel in the tsv file ...
found one pos rel in the tsv file ...
found one pos rel in the tsv file ...
found one pos rel in the tsv file ...
found one pos rel in the tsv file ...
found one pos rel in the tsv file ...
found one pos rel in the tsv file ...
found one pos rel in the tsv file ...
found one pos rel in the tsv file ...
found one pos rel in the tsv file ...
found one pos rel in the tsv file ...
found one pos rel in the tsv file ...


In [7]:
# %history

In [8]:
# pos_rel[0]

In [ ]:
pos_rel

In [10]:
def find_loc(a, b):
    # a: long one
    a = re.sub('[^0-9a-zA-Z]', ' ', a)
    b = re.sub('[^0-9a-zA-Z]', ' ', b)
    a_clean = ''.join(a.split())
    b_clean = ''.join(b.split())
    index = a_clean.find(b_clean)
    s, out = 0, []
    for i, char in enumerate(a):
        if char == " " or char == "\t":
            continue 
        s+=1 
        if s == index + 1: 
            out.append(i)           
        if s == index + len(b_clean):
            out.append(i)
    if len(out) == 2:
        start, end = out[0], out[1]+1
        return start, end
    else: 
        return None, None
final_pos_lst= []
for pos in pos_rel: 
    context = " ".join(pos["context"])

    ment1 = pos["arg1"]["mention"]
    ment1
    ment2 =pos["arg2"]["mention"]
    final_pos_lst.append({
        "filename" :pos["filename"],
        "statement" :pos["statement"],
        "num_pre9_tokens":pos["num_pre9_tokens"],
        "rel_type":pos["rel_type"],
        "label":pos["label"],
        "arg1":{
            "mention":pos["arg1"]["mention"],
            "nth_sent":pos["arg1"]["nth_sent"],
            "start":find_loc(context,ment1)[0],
            "end":find_loc(context,ment1)[1]
        },
        "arg2":{
            "mention":pos["arg2"]["mention"],
            "nth_sent":pos["arg2"]["nth_sent"],
            "start":find_loc(context,ment2)[0],
            "end":find_loc(context,ment2)[1]
        },
        "context":" ".join(pos["context"])

    })


In [ ]:
final_pos_lst

In [ ]:
neg_lst

In [13]:
final_neg_lst = []

for neg in neg_lst:
    context = neg["context"]

    ment1 = neg["arg1"]["mention"]
    ment2 =neg["arg2"]["mention"]
    final_neg_lst.append({
        "filename" :neg["filename"],
        "statement" :neg["statement"],
        "num_pre9_tokens":neg["num_pre9_tokens"],
        "rel_type":"none",
        "label":"negative",
        "arg1":{
            "mention":neg["arg1"]["mention"],
            "nth_sent":neg["arg1"]["nth_sent"],
            "start":find_loc(context,ment1)[0],
            "end":find_loc(context,ment1)[1]
        },
        "arg2":{
            "mention":neg["arg2"]["mention"],
            "nth_sent":neg["arg2"]["time_nth_sent"],
            "start":find_loc(context,ment2)[0],
            "end":find_loc(context,ment2)[1]
        }, "context":context

    })

    final_neg_lst


In [14]:
print(len(final_pos_lst), len(final_neg_lst)) #113, 101

113 22


In [15]:
both_lst = final_pos_lst+final_neg_lst

file = f"/users/the/NER_MTB/timeset/my_code_result/input_data/{cancer}_{mode}_pos_neg_for_nli.json"
with open(file, "w") as outfile: 
    json.dump(both_lst, outfile, indent=4)

In [16]:
# both_rel = []
# for item in pos_rel: 
#     if len(item["arg1"]) != 0 or len(item["mention"]):
#         both_rel.append(item)

# for item in both_rel:
#     # Extract arg1 and arg2 mention indices from the context
#     context = item["context"][0]  # Assuming context is a list with one element
#     arg1_start = context.find(item["arg1"]["mention"])
#     arg1_end = arg1_start + len(item["arg1"]["mention"]) if arg1_start != -1 else -1
#     arg2_start = context.find(item["arg2"]["mention"])
#     arg2_end = arg2_start + len(item["arg2"]["mention"]) if arg2_start != -1 else -1
    
#     # Add start_ind and end_ind attributes to the dictionary
#     item["arg1"]["start"] = arg1_start
#     item["arg1"]["end"] = arg1_end
#     item["arg2"]["start"] = arg2_start
#     item["arg2"]["end"] = arg2_end

%history

In [17]:
# %history

In [18]:
# merged = df.merge(new_df_filtered, on=['DCT', 'patient_id', 'chemo_text', 'chemo_annotation_id',
#        'normed_timex', 'timex_annotation_id', 'tlink', 'note_name',
#        'tlink_inst'], how="outer", indicator=True)
# unique_to_df1 = merged[merged['_merge'] == 'left_only']
# unique_to_df2 = merged[merged['_merge'] == 'right_only']
# # print("Rows unique to df1:")
# # print(unique_to_df1)

# print("\nRows unique to df2:")
# print(unique_to_df2)

3. generate candiatate pair from 2, mark the pair as negative if it is not at 1 (need index to compare)

In [19]:
# none_count = final_df["nth_sent"].isna().sum()
# total_rows = len(final_df)
# percentage_none = (none_count / total_rows) * 100 #0.32%
# print(f"Percentage of rows with None in 'nth_sent': {percentage_none:.2f}%")
# final_df.shape #(2182, 24)
# same_sent_df = final_df[abs(final_df["nth_sent"].astype(int) - final_df["time_nth_sent"].astype(int)) <= 1]
# print(same_sent_df.shape) #(321, 17)
# same_sent_df.to_csv("./temp_same.tsv", sep="\t", index=False)

In [20]:
# test_df = final_df

In [21]:
# test_df = test_df.dropna()

In [22]:
# neg_lst[0]

In [23]:
# same_sent_df.columns

In [24]:

'''
note_dir = os.path.join("/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/input/change_n2space", \
                        f"input_{cancer}_{mode}", "Patient_Notes", f"{cancer}", f"{mode}")

def _get_string(row):
    return row[0]

tf_label_lst = []
pos_cnt, neg_cnt = 0, 0
for ind, row in same_sent_df[:10].iterrows():
    neg_ins = defaultdict(dict)

    pat_id = row["note_name"].split("_")[0]
    filename = "_".join([row["note_name"].split("_")[1], row["note_name"].split("_")[2]])
    # print(f"~~~~{pat_id}, {filename}~~~~")
    pat_filename = f"{pat_id}_{filename}.txt"
    note_sub_dir = os.path.join(note_dir, pat_id)
    raw_note_file = os.path.join(note_sub_dir, pat_filename)
    try:
        with open(raw_note_file, "r") as infile: 
            lines = infile.readlines()
        lines = lines[9:]
    except:
        print(raw_note_file)
        continue
    clean_note = "".join(lines)
    doc = spacy_better(clean_note)
    seindexs_single_note, sent_lst = _get_seindexs_single_note(clean_note, spacy_better)
    
    num_pre9_tokens = int(gold_dct[pat_id][filename]["num_pre9_tokens"])
    df_chemo_sid, df_chemo_eid = row["chemo_tlinkinst_start"], row["chemo_tlinkinst_end"]
    df_time_sid, df_time_eid = row["time_tlinkinst_start"], row["time_tlinkinst_end"]
    df_chemo_sid, df_chemo_eid = df_chemo_sid+num_pre9_tokens, df_chemo_eid+num_pre9_tokens
    df_time_sid, df_time_eid = int(df_time_sid)+num_pre9_tokens, int(df_time_eid)+num_pre9_tokens
    df_nth_sent, df_time_nth_sent = row["nth_sent"], row["time_nth_sent"]
    ith_start, ith_end = row["ith_start"], row["ith_end"]
    time_ith_start, time_ith_end = row["time_ith_start"], row["time_ith_end"]

    chemo_ment, time_ment = row["chemo_text"], row["raw_time"]
    if filename not in gold_dct[pat_id].keys(): 
        neg_cnt +=1
        neg_ins["filename"] = row["note_name"]
        neg_ins["statement"] = "Does the arg1 and arg2 has relationship?"
        neg_ins["num_pre9_tokens"] = num_pre9_tokens
        neg_ins["rel_type"] = None
        neg_ins["label"] = "negative"

        neg_ins["arg1"] = {
            "mention":chemo_ment,
            "start_paper_ind":df_chemo_sid.astype(int),
            "end_paper_ind":df_chemo_eid.astype(int),
            "nth_sent":df_nth_sent.astype(int)
        }
        neg_ins["arg2"] = {
            "mention":time_ment,
            "start_paper_ind":df_time_sid.astype(int),
            "end_paper_ind":df_time_eid.astype(int),
            "nth_sent":df_time_nth_sent.astype(int)
        }
        # chemo_nth_sent, time_nth_sent = row["nth_sent"], row["time_nth_sent"]
        # if int(chemo_nth_sent) > int(time_nth_sent):
        #     start_nth_sent = time_nth_sent
        #     end_nth_sent = chemo_nth_sent
        # else: 
        #     start_nth_sent = chemo_nth_sent
        #     end_nth_sent = time_nth_sent
        row["match_sents"] = row["match_sents"].apply(_get_string)
        neg_ins["context"] = row["match_sents"]
        both_rel.append(neg_ins)
    else: 
        true_source_id, true_target_id = gold_dct[pat_id][filename]["chemo_time_rel"]["source_id"], gold_dct[pat_id][filename]["chemo_time_rel"]["target_id"]
        
        true_pair = []
        for i, j in zip(true_source_id, true_target_id):
            true_pair.append(tuple([i, j]))

        row_pair = tuple([row["timex_annotation_id"], row["chemo_annotation_id"]])
        if any(sorted(row_pair) == sorted(s) for s in true_pair):
            print(row_pair, true_pair)
            continue
    

        true_pair_span = []
        if pat_id not in gold_dct.keys():
            tf_label_lst.append("negative")
        else: 
            if filename not in gold_dct[pat_id].keys():
                tf_label_lst.append("negative")
            else: 
                for true_pair in zip(gold_dct[pat_id][filename]["chemo_time_rel"]["source_id"], gold_dct[pat_id][filename]["chemo_time_rel"]["target_id"]):
                    if true_pair[0] in gold_dct[pat_id][filename]["pair_wise"]["chemo"]["ment_id"]: 
                        chemo_span = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["span"][gold_dct[pat_id][filename]["pair_wise"]["chemo"]["ment_id"].index(true_pair[0])]
                        time_span = gold_dct[pat_id][filename]["pair_wise"]["time"]["span"][gold_dct[pat_id][filename]["pair_wise"]["time"]["ment_id"].index(true_pair[1])]
                    else: 
                        chemo_span = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["span"][gold_dct[pat_id][filename]["pair_wise"]["chemo"]["ment_id"].index(true_pair[1])]
                        time_span = gold_dct[pat_id][filename]["pair_wise"]["time"]["span"][gold_dct[pat_id][filename]["pair_wise"]["time"]["ment_id"].index(true_pair[0])]
                    # print(chemo_span, time_span)
                    true_pair_span.append((chemo_span, time_span))
                if ([df_chemo_sid, df_chemo_eid],[df_time_sid, df_time_eid]) in true_pair_span or \
                ([df_time_sid, df_time_eid], [df_chemo_sid, df_chemo_eid]) in true_pair_span:
                    pos_cnt += 1
'''


'\nnote_dir = os.path.join("/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/input/change_n2space",                         f"input_{cancer}_{mode}", "Patient_Notes", f"{cancer}", f"{mode}")\n\ndef _get_string(row):\n    return row[0]\n\ntf_label_lst = []\npos_cnt, neg_cnt = 0, 0\nfor ind, row in same_sent_df[:10].iterrows():\n    neg_ins = defaultdict(dict)\n\n    pat_id = row["note_name"].split("_")[0]\n    filename = "_".join([row["note_name"].split("_")[1], row["note_name"].split("_")[2]])\n    # print(f"~~~~{pat_id}, {filename}~~~~")\n    pat_filename = f"{pat_id}_{filename}.txt"\n    note_sub_dir = os.path.join(note_dir, pat_id)\n    raw_note_file = os.path.join(note_sub_dir, pat_filename)\n    try:\n        with open(raw_note_file, "r") as infile: \n            lines = infile.readlines()\n        lines = lines[9:]\n    except:\n        print(raw_note_file)\n        continue\n    clean_note = "".join(lines)\n    doc = spacy_better(clean_note)\n    seindexs_single_note, s

In [25]:
# a = ('1@e@patient47_report034_NOTE@system', '2@e@patient47_report034_NOTE@system')
# b = [('2@e@patient47_report034_NOTE@system', '1@e@patient47_report034_NOTE@system'), ('1@e@patient30_report006_RAD@system', '5@e@patient30_report006_RAD@system')]

# if any(sorted(a) == sorted(s) for s in b):
#     print("yes")
# else:
#     print("no")

In [26]:
                # else: 
                    # neg_ins["filename"] = row["note_name"]
                    # neg_ins["statement"] = "Does the arg1 and arg2 has relationship?"
                    # neg_ins["num_pre9_tokens"] = num_pre9_tokens
                    # neg_ins["rel_type"] = None
                    # neg_ins["label"] = "negative"

                    # neg_ins["arg1"] = {
                    #     "mention":chemo_ment,
                    #     "start_paper_ind":df_chemo_sid,
                    #     "end_paper_ind":df_chemo_eid,
                    #     "nth_sent":df_nth_sent,
                    #     "start":ith_start,
                    #     "end": ith_end
                    # }
                    # neg_ins["arg2"] = {
                    #     "mention":time_ment,
                    #     "start_paper_ind":df_time_sid,
                    #     "end_paper_ind":df_time_eid,
                    #     "nth_sent":df_time_nth_sent,
                    #     "start":time_ith_start,
                    #     "end":time_ith_end
                    # }
                    # print(type(df_chemo_sid), df_chemo_sid)
                    # neg_ins["filename"] = row["note_name"]
                    # neg_ins["statement"] = "Does the arg1 and arg2 has relationship?"
                    # neg_ins["num_pre9_tokens"] = num_pre9_tokens
                    # neg_ins["rel_type"] = None
                    # neg_ins["label"] = "negative"

                    # neg_ins["arg1"] = {
                    #     "mention":chemo_ment,
                    #     "start_paper_ind":df_chemo_sid.astype(int),
                    #     "end_paper_ind":df_chemo_eid.astype(int),
                    #     "nth_sent":df_nth_sent.astype(int)
                    # }
                    # neg_ins["arg2"] = {
                    #     "mention":time_ment,
                    #     "start_paper_ind":df_time_sid.astype(int),
                    #     "end_paper_ind":df_time_eid.astype(int),
                    #     "nth_sent":df_time_nth_sent.astype(int)
                    # }

                    # row["match_sents"] = row["match_sents"].apply(_get_string)
                    # neg_ins["context"] = row["match_sents"]
                    # both_rel.append(neg_ins)

# for item in both_rel:
#     if item["label"] == "negative":
#         # Extract arg1 and arg2 mention indices from the context
#         context = item["context"]  # Assuming context is a list with one element
#         arg1_start = context.index(item["arg1"]["mention"])
#         arg1_end = arg1_start + len(item["arg1"]["mention"]) #if arg1_start != -1 else -1
#         arg2_start = context.index(item["arg2"]["mention"])
#         arg2_end = arg2_start + len(item["arg2"]["mention"]) #if arg2_start != -1 else -1
        
#         # Add start_ind and end_ind attributes to the dictionary
#         item["arg1"]["start"] = arg1_start
#         item["arg1"]["end"] = arg1_end
#         item["arg2"]["start"] = arg2_start
#         item["arg2"]["end"] = arg2_end

# with open(f"./{cancer}_{mode}_input.json", 'w') as outfile:
#     json.dump(both_rel, outfile, indent=4)

# print(neg_cnt, pos_cnt) 
# two sent range: 595 225

In [27]:
# from spacy.tokens import Span
# for index, item in enumerate(pos_rel):
#     if isinstance(item, Span):
#         print(f"Span object found at index {index}")

In [28]:
# cnt = 0
# for item in pos_rel: 
#     if len(item["arg1"]) == 0:
#         cnt+=1
# cnt

In [29]:
# len(pos_rel)

In [30]:
# both_rel = []
# for item in pos_rel: 
#     if len(item["arg1"]) != 0:
#         both_rel.append(item)
# len(both_rel)

In [31]:
# for item in both_rel: 
#     if len(item["context"]) == 0:
#         print(item)


In [32]:
# final_rel = []
# for item in both_rel: 
#     if len(item["context"]) != 0:
#         final_rel.append(item)
# len(final_rel)

In [33]:
# # Assuming data_list is your list of dictionaries
# for item in final_rel:
#     # Extract arg1 and arg2 mention indices from the context
#     context = item["context"][0]  # Assuming context is a list with one element
#     arg1_start = context.find(item["arg1"]["mention"])
#     arg1_end = arg1_start + len(item["arg1"]["mention"]) if arg1_start != -1 else -1
#     arg2_start = context.find(item["arg2"]["mention"])
#     arg2_end = arg2_start + len(item["arg2"]["mention"]) if arg2_start != -1 else -1
    
#     # Add start_ind and end_ind attributes to the dictionary
#     item["arg1"]["start"] = arg1_start
#     item["arg1"]["end"] = arg1_end
#     item["arg2"]["start"] = arg2_start
#     item["arg2"]["end"] = arg2_end

# with open(f'./{cancer}_{mode}.json', 'w') as f:
#     json.dump(final_rel, f, indent=4) 
# len(final_rel)

In [34]:
# both_rel